## Essential Imports

In [ ]:
import tensorflow as tf
from keras.regularizers import l2
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Essential Constants

In [ ]:
train_path = '/content/drive/My Drive/DM Course Project/Temp data(spec images)/temp data(spec images)/train'
test_path = '/content/drive/My Drive/DM Course Project/Temp data(spec images)/temp data(spec images)/test'

# useful for getting number of classes
folders = glob('/content/drive/My Drive/DM Course Project/Temp data(spec images)/temp data(spec images)/train/*')

num_classes=len(folders)

# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [ ]:
print(num_classes)

2


## Creating a new model

In [ ]:
model = Sequential()
model.add(VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(1, kernel_regularizer=l2(0.01),activation='linear')) # Final Layer is SVM Layer
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer='adam', loss='hinge', metrics=['accuracy',tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.Hinge(),tf.keras.metrics.BinaryCrossentropy()])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten_4 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               3211392   
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 129       
Total params: 23,235,905
Trainable params: 3,211,521
Non-trainable params: 20,024,384
_________________________________________________________________


## Setting up training and testing data

In [ ]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
train_set = data_generator.flow_from_directory(train_path,target_size = (224, 224),batch_size=10,class_mode = 'binary')
test_set = data_generator.flow_from_directory(test_path,target_size = (224, 224),batch_size=10,class_mode = 'binary')

Found 3000 images belonging to 2 classes.
Found 1400 images belonging to 2 classes.


In [ ]:
print(test_set)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
# checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=20, verbose=1, mode='auto')

In [ ]:
r = model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set)
#   callbacks=[early]
)

f=open('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-SI-VGG19-stats.txt','w')
f.write(str(r.history))
f.close()

model.save('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-SI-VGG19-model.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
300/300 [==============================] - 2698s 9s/step - loss: 3.9181 - accuracy: 0.6270 - mean_absolute_error: 9.8275 - mean_squared_error: 202.4597 - hinge: 3.9020 - binary_crossentropy: 5.4627 - val_loss: 1.0011 - val_accuracy: 0.7236 - val_mean_absolute_error: 5.6254 - val_mean_squared_error: 49.5252 - val_hinge: 0.9891 - val_binary_crossentropy: 3.7057
Epoch 2/20
300/300 [==============================] - 66s 219ms/step - loss: 0.9914 - accuracy: 0.7823 - mean_absolute_error: 6.0233 - mean_squared_error: 57.3412 - hinge: 0.9798 - binary_crossentropy: 2.9799 - val_loss: 0.6344 - val_accuracy: 0.8529 - val_mean_absolute_error: 6.5700 - val_mean_squared_error: 65.4401 - val_hinge: 0.6241 - val_binary_crossentropy: 1.9753
Epoch 3/20
300/300 [==============================] - 66s 219ms/step - loss: 0.6255 - accuracy: 0.8458 - mean_absolute_error: 6.3115 - mean_squared_error: 65.2419 - hinge: 0.6156 - binary_crossentropy: 2.0766 - val_loss: 0.5772 - val_accuracy: 0.8643 - v